# Local RAG Demo — Condominium Regulations

This notebook demonstrates a local Retrieval-Augmented Generation (RAG) pipeline
using Ollama and ChromaDB.

## Prerequisites

- Ollama running locally
- Required models pulled:
  - mistral:7b-instruct
  - nomic-embed-text
- Python environment set up

## Pipeline

The solution has the following stages:

1. Data Ingestion
2. Data Embedding
3. Vectorization
4. Retrieval
5. Augmentation
6. Response Generation

## File Structure

This is the project's file structure:

rag/
  README.md
  jose_rag.ipynb          # Jupyter notebook
  src/
    ingest.py
    ollama_embed.py
    vectordb.py
    retrieve_chroma.py
    rag.py
    generate.py
    v1/                   #first version of embed and index phases
  data/
    Condominium regulations - chapter 1.txt
    Condominium regulations - chapter 10.txt
    Condominium regulations - chapter 2.txt
    Condominium regulations - chapter 3.txt
    Condominium regulations - chapter 4.txt
    Condominium regulations - chapter 5.txt
    Condominium regulations - chapter 6.txt
    Condominium regulations - chapter 7.txt
    Condominium regulations - chapter 8.txt
    Condominium regulations - chapter 9.txt
  chroma_db/               # will be created upon first run of the solution

***Please see the README.md file for additional details***

## Testing if the Local LLM works

The next two cells are just to test Ollama (local LLM) is working.
You can change the "prompt" below and rerun the cell to confirm.

In [11]:
import requests
import json

MODEL = "mistral:7b-instruct"  # change if you pulled a different one

url = "http://localhost:11434/api/generate"
payload = {
    "model": MODEL,
    "prompt": "Can you make a summary of all fines in a condominium?",
    "stream": False
}

resp = requests.post(url, json=payload, timeout=120)
resp.raise_for_status()

data = resp.json()
print("Model responded:", data["response"])

Model responded:  In a condominium, fines are typically imposed for violations of the rules and regulations set by the Condominium Association. Here's a general summary:

1. Late Payment Fines: If an owner fails to pay their monthly or special assessments on time, they may be subjected to late payment fines.

2. Rule Violation Fines: Fines can also be imposed for violations of the condominium rules and regulations, such as parking in unassigned spaces, keeping pets without permission, making modifications without approval, or causing disturbances that disrupt other residents.

3. Maintenance and Repair Fines: If an owner fails to maintain their unit according to the association's guidelines or causes damage to common areas, they may be fined for these violations.

4. Fee Assessments: In some cases, the Condominium Association may impose a fee assessment on all owners to cover unexpected costs, such as repairs to the building or grounds, legal fees, or insurance premium increases.

5. L

## Step 1 — Load and Chunk Documents

Documents are loaded from disk and split into overlapping chunks.
Low-quality chunks are filtered to reduce retrieval noise.

Files: 
- src/ingest.py

In [1]:
from src.ingest import load_documents, chunk_documents

docs = load_documents("data")
chunks = chunk_documents(docs, chunk_size=800, overlap=100)

print("Documents loaded:", len(docs))
print("Total chunks:", len(chunks))

print("\nSample chunks:")
for c in chunks[:3]:
    print("----")
    print("chunk_id:", c.chunk_id)
    print("source:", c.source)
    print("chunk_index:", c.chunk_index)
    print("text preview:", c.text[:150], "...")


Documents loaded: 10
Total chunks: 24

Sample chunks:
----
chunk_id: Condominium regulations - chapter 1.txt::chunk_0
source: Condominium regulations - chapter 1.txt
chunk_index: 0
text preview: ### **INTERNAL REGULATIONS FOR COEXISTENCE AND ADMINISTRATION**
**FOR THE VERTICAL CONDOMINIUM "TRIVENTO III"**
**Last Updated:** December 2025
**Effe ...
----
chunk_id: Condominium regulations - chapter 1.txt::chunk_1
source: Condominium regulations - chapter 1.txt
chunk_index: 1
text preview: uption:** After the third month of non-payment, the potable water supply will be cut off and, in extreme cases, access to common areas and electronic  ...
----
chunk_id: Condominium regulations - chapter 1.txt::chunk_2
source: Condominium regulations - chapter 1.txt
chunk_index: 2
text preview: box; b) registered email; c) message through the official condominium application (if one exists). They will be considered received 48 hours after bei ...


(Optional) If you want, you can print the number of documents and chunks below:

In [ ]:
print("Documents:", len(docs))
print("Chunks:", len(chunks))

## Step 2 — Build Vector Index

Chunks are embedded using a local Ollama embedding model and stored
in a persistent ChromaDB collection.

***Note that the number of actual indexed chunks might be lower from the step before due to data cleanup exclusions.*** 

Files: 
- src/ingest.py
- src/ollama_embed.py
- src/vectordb.py

In [2]:
from src.ingest import load_documents, chunk_documents
from src.ollama_embed import ollama_embed
from src.vectordb import get_chroma_client, get_or_create_collection, upsert_chunks

docs = load_documents("data")
chunks = chunk_documents(docs, chunk_size=800, overlap=100)

# embed all chunks via Ollama
chunk_embeddings = ollama_embed([c.text for c in chunks], model="nomic-embed-text")

client = get_chroma_client("chroma_db")
col = get_or_create_collection(client, "condo_rules")

upsert_chunks(col, chunks, chunk_embeddings)

print("Indexed chunks:", len(chunks))


Indexed chunks: 24


## Step 3 — Retrieve Relevant Context

Given a user question, we embed the query and retrieve the top-K (5 in the example below)
most similar chunks from the vector database.

With Chroma, the **difference** between the query vector and each stored result is calculated.
For that reason, each **"score"** represents how different each chunk is from the query. The smaller the score number, the closer the chunk is to the query.

In [3]:
from src.ollama_embed import ollama_embed
from src.vectordb import get_chroma_client, get_or_create_collection
from src.retrieve_chroma import retrieve_top_k_chroma

client = get_chroma_client("chroma_db")
col = get_or_create_collection(client, "condo_rules")

query = "What are the Quiet Hours?"
q_emb = ollama_embed([query], model="nomic-embed-text")[0]

results = retrieve_top_k_chroma(col, top_k=5, query_embedding=q_emb)

for r in results:
    print("----")
    print("score:", round(r.distance, 4))
    print("chunk_id:", r.chunk_id)
    print("source:", r.source)
    print("preview:", r.text[:180].replace("\n", " "), "...")


----
score: 0.884
chunk_id: Condominium regulations - chapter 2.txt::chunk_0
source: Condominium regulations - chapter 2.txt
preview: ### **INTERNAL REGULATIONS FOR COEXISTENCE AND ADMINISTRATION** **FOR THE VERTICAL CONDOMINIUM "TRIVENTO III"** **Last Updated:** December 2025 **Effective Date:** From January 1,  ...
----
score: 0.8912
chunk_id: Condominium regulations - chapter 5.txt::chunk_0
source: Condominium regulations - chapter 5.txt
preview: ### **INTERNAL REGULATIONS FOR COEXISTENCE AND ADMINISTRATION** **FOR THE VERTICAL CONDOMINIUM "TRIVENTO III"** **Last Updated:** December 2025 **Effective Date:** From January 1,  ...
----
score: 0.9326
chunk_id: Condominium regulations - chapter 8.txt::chunk_0
source: Condominium regulations - chapter 8.txt
preview: ### **INTERNAL REGULATIONS FOR COEXISTENCE AND ADMINISTRATION** **FOR THE VERTICAL CONDOMINIUM "TRIVENTO III"** **Last Updated:** December 2025 **Effective Date:** From January 1,  ...
----
score: 0.9404
chunk_id: Condominium 

## Step 4 — Prompt Augmentation

Retrieved context is injected into a structured prompt that enforces:
- grounding
- citations
- no hallucination

In [12]:
from src.ollama_embed import ollama_embed
from src.vectordb import get_chroma_client, get_or_create_collection
from src.retrieve_chroma import retrieve_top_k_chroma
from src.rag import build_augmented_prompt

client = get_chroma_client("chroma_db")
col = get_or_create_collection(client, "condo_rules")

query = "Can you make a summary of all fines in a condominium?"
q_emb = ollama_embed([query], model="nomic-embed-text")[0]

results = retrieve_top_k_chroma(col, top_k=5, query_embedding=q_emb)

prompt = build_augmented_prompt(query, results)  # results = retrieval results from Chroma
print(prompt[:2000])
print("\nPrompt length:", len(prompt))
print("Retrieved chunks:", len(results))


You are a helpful assistant answering questions about condominium regulations.
Use ONLY the provided CONTEXT to answer.
If the answer is not in the CONTEXT, say: "I don't know based on the provided regulations."
ALWAYS include citations.
Every sentence that states a rule, time window, restriction, fine, or requirement MUST end with a citation
in square brackets using the chunk_id, e.g. [Condominium regulations - chapter 5.txt::chunk_2].
Be concise and factual.

QUESTION:
Can you make a summary of all fines in a condominium?

CONTEXT:
[Condominium regulations - chapter 4.txt::chunk_1] at invade the privacy of third parties. * 14.3. **Personal Data:** The condominium will be governed by the Personal Data Protection Act. The information of owners and tenants (telephone, email, license plate) will be treated confidentially and used only for administrative purposes.  **ARTICLE 15°.- OFFICIAL APPLICATION AND DIGITAL COMMUNICATIONS.** * 15.1. The condominium will implement an official applica

## Step 5 — Generate Answer

The augmented prompt is sent to a local LLM via Ollama.

In [14]:
from src.generate import generate_with_ollama
answer = generate_with_ollama(prompt, model="mistral:7b-instruct", temperature=0.2)
print(answer)


Fines in a condominium include:

1. Non-compliance with privacy regulations (invading third parties' privacy): Fine not specified [Condominium regulations - chapter 4.txt::chunk_1]
2. Failure to update contact details in the official application/inbox: No fine specified, but it may lead to communication issues [Condominium regulations - chapter 4.txt::chunk_15.2]
3. Installation of private cables, antennas or routers affecting the facade or common areas without permission: Fine not specified [Condominium regulations - chapter 4.txt::chunk_16.1]
4. Non-payment of potable water supply for more than three months: Water supply cut off, access to common areas and electronic systems restricted; reconnection requires full payment of the outstanding balance plus a disconnection/reconnection fee of US$ 150 [Condominium regulations - chapter 1.txt::chunk_1]
5. Violation of video surveillance or digital access system: Fine not specified [Condominium regulations - chapter 4.txt::chunk_14.2]
6. Exc

In [ ]:
## Why RAG?

Without retrieval, an LLM may guess policy details.
With RAG, answers are grounded in the actual regulations and include citations.

You may want to try the same query in the first cell of this notebook to compare the answers.